In [1]:
%%time
import numpy as np
import ast
from datetime import datetime
import glob
import gzip
import ipaddress
import os
import pandas as pd
from pandarallel import pandarallel
from joblib import Parallel, delayed
from local_utils import *
import sys
import logging

so = open('apews.log', 'w', 10)
sys.stdout.echo = so
sys.stderr.echo = so

get_ipython().log.handlers[0].stream = so
get_ipython().log.setLevel(logging.INFO)

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 60 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
CPU times: user 664 ms, sys: 194 ms, total: 858 ms
Wall time: 2.85 s


In [2]:
blacklist_dir = '/data/data-ec/data/blacklists'
exploded_ranges_file = 'csv/all_ranges_exploded.csv'
apews_output_dir = '/data/data-sh/data/data.external-scans/apews'

In [3]:
# exploded = pd.read_csv('csv/all_ranges_over_time_gaps_filled.csv')
# del exploded['ripe_ranges']

# def process_asdf(row):
#     row['ripe_names'] = ast.literal_eval(row['ripe_names'])
#     row['Ranges'] =  [ ipaddress.IPv4Network(x) for x in ast.literal_eval(row['Ranges']) ]
#     return row

# exploded = exploded.parallel_apply(process_asdf, axis=1)

# cols = list(exploded.columns.values)
# cols.remove('Ranges')
# cols.remove('ripe_names')

# exploded = exploded.set_index(cols).apply(pd.Series.explode).reset_index()
# exploded.to_csv(exploded_ranges_file, index=False)


In [4]:
def process_cols(row):
#     row['date'] = pd.to_datetime(row['date'], format='%Y-%m-%d')
    row['Ranges'] = ipaddress.IPv4Network(row['Ranges'])
    row['ripe_names'] = '' if pd.isna(row['ripe_names']) else row['ripe_names']
    row['ripe_names'] = row['ripe_names'] if row['ripe_names'] != '' else row['org']
#     row['md5'] = md5dict.get(row['date'], '')
    return row

with open(f'{apews_output_dir}/date-md5.txt', 'r') as f:
    files_md5 = ast.literal_eval(f.read())
md5dict = dict((x, y) for x, y in files_md5)
    
exploded = pd.read_csv(exploded_ranges_file, dtype={'ripe_names': 'string'})
exploded['md5'] = exploded['date'].parallel_apply(lambda x: md5dict.get(x, ''))
exploded = exploded.parallel_apply(lambda x: process_cols(x), axis=1)
exploded

,date,customer,Organization,first,last,size,ripe_ranges_size,Ranges,ripe_names,md5
0,2014-02-22,AFAS,AFAS Software BV,2015-07-17,2022-01-19,1024,1024,185.46.180.0/22,nl.afas,068ccdb1abe03ec94f6b71f84f89e56c
1,2014-02-23,AFAS,AFAS Software BV,2015-07-17,2022-01-19,1024,1024,185.46.180.0/22,nl.afas,766ca4236ac6267ad1913e922330f503
2,2014-02-24,AFAS,AFAS Software BV,2015-07-17,2022-01-19,1024,1024,185.46.180.0/22,nl.afas,719df0bf3451d59f0256e1332b746cb2
3,2014-02-25,AFAS,AFAS Software BV,2015-07-17,2022-01-19,1024,1024,185.46.180.0/22,nl.afas,
4,2014-02-26,AFAS,AFAS Software BV,2015-07-17,2022-01-19,1024,1024,185.46.180.0/22,nl.afas,
...,...,...,...,...,...,...,...,...,...,...
20778749,2017-12-19,unit4,Unit4 Business Software B.V.,2015-01-06,2018-01-04,1024,1024,185.125.236.0/22,nl.unit4,fa915d8aebfdcdffbbb1434828ffe595
20778750,2017-12-20,unit4,Unit4 Business Software B.V.,2015-01-06,2018-01-04,1024,1024,185.125.236.0/22,nl.unit4,fa915d8aebfdcdffbbb1434828ffe595
20778751,2017-12-21,unit4,Unit4 Business Software B.V.,2015-01-06,2018-01-04,1024,1024,185.125.236.0/22,nl.unit4,fa915d8aebfdcdffbbb1434828ffe595
20778752,2017-12-22,unit4,Unit4 Business Software B.V.,2015-01-06,2018-01-04,1024,1024,185.125.236.0/22,nl.unit4,fa915d8aebfdcdffbbb1434828ffe595


In [5]:
def get_file_location_dict(directory):
    files = glob.glob(f'{directory}/*.csv')
    # extract md5 string
    files = { file.replace(f'{directory}/', '').replace('.csv', ''): file for file in files }
    return files

In [6]:
def get_ips_and_ranges_set_tuple(filepath):
    ips = []
    ip_ranges = []
    exclude = []
    exclude_ranges = []
    with open(filepath, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
        
    for line in lines:
        if line.startswith('$') or line.startswith(':'):
            continue
        line = line.strip()
        parts = line.split(' ')
        ip = parts[0]
        if '/' in ip:
            if ip.startswith('!'):
                try:
                    exclude_ranges.append(ipaddress.IPv4Network(ip[1:], strict=False))
                except:
                    r = ip[1:ip.index('/')]
                    r = r.split('.')
                    r = [x if x != '' else '0' for x in r]

                    # files contain tons of (sort of fixable) errors,
                    # but an IP address with >4 octets we will just skip
                    if len(r) > 4:
                        continue

                    while len(r) < 4:
                        r.append('0')

                    r = '.'.join(r)
                    mask = ip[ip.index('/'):]
                    try:
                        ip_ranges.append(ipaddress.IPv4Network(r + mask, strict=False))
                    except:
                        mask = mask[0:mask.index('/')]
                        ip_ranges.append(ipaddress.IPv4Network(r + mask, strict=False))
            else:
                try:
                    ip_ranges.append(ipaddress.IPv4Network(ip, strict=False))
                except:
                    print(ip)
                    r = ip[0:ip.index('/')]
                    r = r.split('.')
                    r = [x if x != '' else '0' for x in r]

                    # files contain tons of (sort of fixable) errors,
                    # but an IP address with >4 octets we will just skip
                    if len(r) > 4:
                        continue

                    while len(r) < 4:
                        r.append('0')

                    r = '.'.join(r)
                    mask = ip[ip.index('/'):]
                    try:
                        ip_ranges.append(ipaddress.IPv4Network(r + mask, strict=False))
                    except:
                        mask = mask[0:mask.index('/')]
                        ip_ranges.append(ipaddress.IPv4Network(r + mask, strict=False))
        else:
            if ip.startswith('!'):
                exclude.append(ip[1:])
            else:
                ips.append(ip)
                    
    ips = set(ips) - set(exclude)
    remove = []
    for ip in ips:
        try:
            ip_addr = ipaddress.IPv4Address(ip)
        except:
            try:
                ip_addr = ipaddress.IPv4Address(ip.lstrip('0'))
            except:
                continue
#         for r in ip_ranges:
#             if ip_addr in r:
#                 remove.append(ip)
        for r in exclude_ranges:
            if ip_addr in r:
                remove.append(ip)
                
    ips = ips - set(remove)
    
    resulting_ranges = []
    for r in ip_ranges:
        removed = False
        for er in exclude_ranges:
            if r.supernet_of(er):
                res_ranges = r.address_exclude(er)
                for rr in res_ranges:
                    resulting_ranges.append(rr)
                removed = True
                
        if not removed:
            resulting_ranges.append(r)
            
    resulting_ranges = set(resulting_ranges)
    return (ips, resulting_ranges)


def get_ip_set(filepath):
    ips = []
    exclude = []
    with gzip.open(filepath, 'rt') as f:
        for line in f:
            if line.startswith('$') or line.startswith(':'):
                continue
            line = line.strip()
            parts = line.split(' ')
            ip = parts[0]
            if not '/' in ip:
                if ip.startswith('!'):
                    exclude.append(ip[1:])
                else:
                    ips.append(ip)
                    
    ips = set(ips)
    exclude = set(exclude)
    return ips - exclude

In [7]:
def extract_features_group(group, filepath):
    md5 = group['md5'].iloc[0]
    print(f'processing {md5}')
    
    if filepath == '':
        group['apews_unique_ip_count'] = pd.Series(pd.array([np.nan for x in group.index], dtype="Int64"),
                                                   index=group.index, name='apews_unique_ip_count')
        group['apews_ranges'] = pd.Series(pd.array([np.nan for x in group.index], dtype="Int64"),
                                                   index=group.index, name='apews_ranges')
        group['apews_malicious_range'] = pd.Series(pd.array([np.nan for x in group.index], dtype="Int64"),
                                                   index=group.index, name='apews_malicious_range')
        return group
    
    (remaining_ips, ranges) = get_ips_and_ranges_set_tuple(filepath)
#     ip_set = get_ip_set(filepath)
    
    result = group.apply(extract_features, args=(remaining_ips, ranges), axis=1)
    return result
    
def extract_features(row, ip_set, ranges):
    # get single IP range from Ranges column, get all IPs in this range
    r = ipaddress.IPv4Network(row['Ranges'])
    # if this range is in malicious ranges, count all IPs in this range as unique malicious IPs
    contained = False
    for bad_range in ranges:
        if bad_range.supernetof(r):
            contained = True
            break
            
    if contained:
        row['apews_unique_ip_count'] = r.num_addresses
        row['apews_ranges'] = str(r)
        row['apews_malicious_range'] = 1
    else:    
        ips = set([ ip.compressed for ip in r ])
        #find union between range and 'malicious' IPs, this is the number of malicious IPs on this date
        union = ips & ip_set
        num_malicious = len(union)
        row['apews_unique_ip_count'] = num_malicious
        row['apews_ranges'] = list(union)
        row['apews_malicious_range'] = 0
        
    return row

In [8]:
file_location_dict = get_file_location_dict(apews_output_dir)

In [9]:
# for date, filepath in file_location_dict.items():
#     (ip_set, ranges) = get_ips_and_ranges_set_tuple(filepath)
#     with open(f'{apews_output_dir}/{date}.csv', 'w') as f:
#         for ip in ip_set:
#             f.write(f'{ip}\n')
#         for r in ranges:
#             f.write(f'{str(r)}\n')

In [10]:
# for date, filepath in file_location_dict.items():
#     ip_set = get_ip_set(filepath)
#     with open(f'{apews_output_dir}/{date}.csv', 'w') as f:
#         for ip in ip_set:
#             f.write(f'{ip}\n')


In [11]:
features = Parallel(n_jobs=20, verbose=1, backend='multiprocessing')(
    delayed(extract_features_group)(group, file_location_dict.get(md5, '')) 
    for i, (md5, group) in enumerate(exploded.groupby('md5'))
)

# features = []
# for i, (date, group) in enumerate(exploded.groupby('date')):
#     filepath = file_location_dict.get(date, '')
#     features.append( extract_features_group(group, filepath) )
#     if i % 250 == 0:
#         print(f'Done task {i}: {date}')

features = pd.concat(features)

[Parallel(n_jobs=20)]: Using backend MultiprocessingBackend with 20 concurrent workers.


processing 
processing 068ccdb1abe03ec94f6b71f84f89e56c
processing 0944a86faf07d14fc1f005918d6d1b44
processing 0ce7ccbace4301d7aead31676890286d
processing 0fe607a670c54e2082112a32386ebd7c
processing 138905d1799e03195e84fe68b4282fd3
processing 16842b5ef118a3348c22ad3d30e8e61e
processing 199befa67554744b226e8513920dfd94
processing 1d89742721bff9e06e709573a5c20c5e
processing 1e67c847c7ef05d6a7381c5bb67edde5

processing 24ba86feb001c40017260a70ce6ffec3processing 251c292e03140d7dfa858b094d8fe989
processing 26198f8d27e7d846cb153b6be11a19e8
processing 26ffaede180338772d511c45612daa83
processing 2aa64b9d43443f6ba8a3486c4d182fbc
processing 2e6535b3719c709460ebd5c920199997
processing 2f8d90974613f7259473ef3082742d03
processing 32e199f1a49398a5cb39bdc8b447ffa7
processing 3495d6dbe71292ba742c07a667f5bf47
processing 383f1d60a0f1cb74ddf503d8a3535d9b
processing 38ea46764e00384a910c6b2bf2279e52
processing 3cf3c02f1fd5230ec2de2c99463e7386
processing 3f52d19fd1923864b465d4e1ce986356
processing 3f582e89c

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:  7.1min


processing 66d4d0abce932e6ac67d6cfaf7c72205
processing 6a8f97953f54701373d5e3e90e7cff12
processing 6a9ca4ead11a7308ee188861971bc5fc
processing 6af8d3336ab79d0bd4b284335cc8b66e
processing 6f75d452b52daf8dad253217c9b15d58
processing 7134b42868438d907e578ea104f56937
processing 719df0bf3451d59f0256e1332b746cb2
processing 73e2fb971ca958688c94b44cbc6d88aa
processing 75d914363c844d15e6bcc8ed2b4b519a
processing 766ca4236ac6267ad1913e922330f503
processing 7989444568bbe613abd48749c8f598af
processing 7b9532297e017bb0c077c0412fa4ef73
processing 7be46db9684996fe8cecbadd4b122330
processing 7c3d4fb1d4e7b3d91494f6cf5d35963a
processing 7d6fcdd85c4f30461c0f40cdb4240235
processing 8083fff2514bd89f8f37553876d12a93
processing 829e0f39753e6743105853e81eca67ae
processing 83c05a2a5809741b13fd5a0980221d7e
processing 83fb1dc64540cc6ba2783224cc96f410
processing 84c566871f7967792f1f120599c5e674
processing 857e287042e526bd274c3722e0d239e2
processing 8fe55f6504eea1c14c4d6134ddb0a6d4
processing 92210b96a980f36439b81

[Parallel(n_jobs=20)]: Done  99 out of  99 | elapsed: 97.8min finished


In [12]:
features[features['apews_unique_ip_count'] > 0]

In [13]:
cols = list(exploded.columns.values)
cols.remove('Ranges')
cols.remove('ripe_names')

In [14]:
features = features.sort_values(['date', 'customer', 'Ranges'])
features['Ranges'] = features['Ranges'].apply(str)

def aggregate_to_list(group):
    return group.tolist() # or whatever other aggregation you want

# columns specified in .agg() should NOT include the groupby columns
imploded = features.groupby(cols).agg({'Ranges': aggregate_to_list,
                                       'ripe_names': aggregate_to_list,
                                       'apews_unique_ip_count': aggregate_to_list,
                                       'apews_ranges': aggregate_to_list,
                                       'apews_malicious_range': aggregate_to_list}).reset_index()

del imploded['md5']

imploded

In [15]:
imploded.to_csv('csv/features/apews_features.csv', index=False)